In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html

### Heart Rate Data

In [11]:
heart_rate = pd.read_csv("HR_001.csv")
heart_rate["datetime"] = pd.to_datetime(heart_rate["datetime"], format="%m/%d/%y %H:%M")
heart_rate["date"] = heart_rate["datetime"].dt.date
heart_rate["hour"] = heart_rate["datetime"].dt.hour
heart_rate["minute"] = heart_rate["datetime"].dt.minute
heart_rate = heart_rate.groupby(['date','hour','minute']).mean()
heart_rate = heart_rate.reset_index()
heart_rate.head()

,date,hour,minute,hr
0,2020-02-13,15,29,85.130500
1,2020-02-13,15,30,92.563167
2,2020-02-13,15,31,97.772833
3,2020-02-13,15,32,99.167833
4,2020-02-13,15,33,88.638333


In [12]:
heart_rate['hr'] = heart_rate[' hr']

In [13]:
heart_rate = heart_rate.drop(columns=' hr')

In [27]:
heart_rate["time"] = heart_rate["hour"] + heart_rate["minute"] / 60
heart_rate.head()

,date,hour,minute,hr,time
0,2020-02-13,15,29,85.130500,15.483333
1,2020-02-13,15,30,92.563167,15.500000
2,2020-02-13,15,31,97.772833,15.516667
3,2020-02-13,15,32,99.167833,15.533333
4,2020-02-13,15,33,88.638333,15.550000


### Calorie Data 

In [18]:
food = pd.read_csv("Food_Log_001.csv")
food = food[['time_begin', 'calorie']]
food["time_begin"] = pd.to_datetime(food["time_begin"] )
food['hour'] = pd.to_datetime(food['time_begin']).dt.hour
food['date'] = pd.to_datetime(food['time_begin']).dt.date
food.head()

,time_begin,calorie,hour,date
0,2020-02-13 18:00:00,456.0,18,2020-02-13
1,2020-02-13 20:30:00,475.0,20,2020-02-13
2,2020-02-13 20:30:00,13.0,20,2020-02-13
3,2020-02-14 07:10:00,120.0,7,2020-02-14
4,2020-02-14 07:10:00,110.0,7,2020-02-14


In [42]:
food = food.groupby(['date','hour']).mean().reset_index()
food.head()

,date,hour,calorie
0,2020-02-13,18,456.0
1,2020-02-13,20,244.0
2,2020-02-14,7,115.0
3,2020-02-14,9,280.0
4,2020-02-14,12,179.0


In [43]:
dates = heart_rate['date'].unique()

### agg

In [44]:
#this needs to be looped per each date
#adjusting time so graph line is smoother and less bunched up together with points
heart_rate["time_rounded"] = (heart_rate["minute"] // 5) * 5  # Round minutes to nearest 5
hr = heart_rate.groupby(["hour", "time_rounded"])["hr"].mean().reset_index()
hr["time"] = hr["hour"] + hr["time_rounded"] / 60
hr["hr_smoothed"] = hr["hr"].rolling(window=10, min_periods=1).mean()
hr.head()

,hour,time_rounded,hr,time,hr_smoothed
0,0,0,62.683187,0.000000,62.683187
1,0,5,61.484713,0.083333,62.083950
2,0,10,59.964167,0.166667,61.377356
3,0,15,59.512838,0.250000,60.911226
4,0,20,58.667754,0.333333,60.462532


## Graph

In [50]:
unique_dates = sorted(heart_rate["date"].unique())

# Initialize Dash App
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Heart Rate and Caloric Intake Over Time"),
    
    # Dropdown to select date
    dcc.Dropdown(
        id="date-dropdown",
        options=[{"label": str(date), "value": str(date)} for date in unique_dates],
        value=str(unique_dates[0]),  # Default to first available date
        clearable=False
    ),

    # Graph Output
    dcc.Graph(id="heart-rate-graph")
])

# Callback to update graph based on selected date
@app.callback(
    Output("heart-rate-graph", "figure"),
    Input("date-dropdown", "value")
)
def update_graph(selected_date):
    selected_date = pd.to_datetime(selected_date).date()  # Convert to date format

    # Filter Data for Selected Date
    selected_hr = heart_rate[heart_rate["date"] == selected_date]
    selected_calories = food[food["date"] == selected_date]

    # Adjust heart rate data (5-minute binning)
    selected_hr["time_rounded"] = (selected_hr["minute"] // 5) * 5  # Round minutes to nearest 5
    hr = selected_hr.groupby(["hour", "time_rounded"])["hr"].mean().reset_index()
    hr["time"] = hr["hour"] + hr["time_rounded"] / 60  # Convert to fractional hour
    hr["hr_smoothed"] = hr["hr"].rolling(window=10, min_periods=1).mean()  # Rolling average smoothing

    # Aggregate calorie intake per hour

    # Create the figure
    fig = go.Figure()

    # Add heart rate as a smooth spline line plot
    fig.add_trace(go.Scatter(
        x=hr["time"], 
        y=hr["hr_smoothed"], 
        mode='lines+markers',
        name='Heart Rate (bpm)',
        line=dict(color='blue', shape='spline', smoothing=1.3),
        marker=dict(size=2, opacity=0.6)
    ))

    # Add calorie intake as a bar chart per hour
    fig.add_trace(go.Bar(
        x=selected_calories["hour"], 
        y=selected_calories["calorie"], 
        name='Caloric Intake',
        yaxis='y2',
        opacity=0.6
    ))

    # Update layout
    fig.update_layout(
        title=f"Heart Rate and Caloric Intake for {selected_date}",
        xaxis=dict(title="Hour", tickmode="linear", dtick=1),
        yaxis=dict(title="Heart Rate (bpm)", side="left", showgrid=False),
        yaxis2=dict(title="Caloric Intake", overlaying="y", side="right", showgrid=False),
        legend=dict(x=0, y=1),
        barmode="overlay"
    )

    return fig

# Run App on a different port (e.g., 8060)
if __name__ == "__main__":
    app.run_server(debug=True, port=8060)

C:\Users\marij\AppData\Local\Temp\ipykernel_13888\150373647.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

